In [2]:
import pandas as pd
import numpy as np
import random 
import csv

from info.info import *

# Estudiantes

In [6]:
estudiantes = pd.DataFrame(columns=['nombre', 'sexo', 'edad', 'RUT', 'correo' , 'universidad', 'telefono','sector', 'carrera', 'año'])

for i in range(15_000):

    # Selección del Nombre y Sexo
    genero = random.randint(0, 2)
    if genero > 0:
        nombre = random.choice(nombres_hombre)
        nombre2 = random.choice(nombres_hombre)
        sexo = 'M'
    else:
        nombre = random.choice(nombres_mujer)
        nombre2 = random.choice(nombres_mujer)
        sexo = 'F'
    apellido = random.choice(apellidos)
    apellido2 = random.choice(apellidos)

    nombre_completo = nombre + ' ' + nombre2 + ', ' + apellido + ' ' + apellido2
   
    # Selección de la edad
    edad = random.randint(18, 30)

    # Selección de la universidad
    u = random.random()
    if   u < 0.2: universidad = 'Universidad de Chile'
    elif u < 0.4: universidad = 'Pontificia Universidad Católica de Chile'
    elif u < 0.5: universidad = 'Universidad de Concepción'
    else:         universidad = random.choice(universidades_chilenas)

    # Creación del Correo
    c = random.random()
    if   c < 0.2: correo = nombre.lower()    + '.' + apellido.lower() + '@' + correos_universidades[universidad].lower()
    elif c < 0.4: correo = nombre[0].lower() + '.' + apellido.lower() + '@' + correos_universidades[universidad].lower()
    elif c < 0.6: correo = nombre.lower()    + '.' + apellido[0].lower() + str(random.randint(0,99)) + '@' + correos_universidades[universidad].lower()
    else:         correo = nombre[0].lower() + '.' + apellido.lower() + str(random.randint(0,99)) + '@' + correos_universidades[universidad].lower()
    

    # creación del RUT
    if   edad < 20: 
        rut = '21' 
        año = random.randint(1,3)
    elif edad < 24:
        rut = '20'
        año = random.randint(2,4)
    elif edad < 28:
        rut = '19'
        año = random.randint(4,6)
    else:           
        rut = '18'
        año = random.randint(6,8)
    rut += '.'+ "".join([str(random.randint(0,9)) for _ in range(3)]) +'.'+ "".join([str(random.randint(0,9)) for _ in range(3)]) +'-'+ random.choice(indice)

    telefono = "+56 9 "+ "".join([str(random.randint(0,9)) for _ in range(4)]) +"-"+ "".join([str(random.randint(0,9)) for _ in range(4)])
    
    # Creación de la carrera
    s = random.random()
    if   s < 0.4: sector = "Ingeniería y Tecnología"
    elif s < 0.6: sector = "Ciencias de la Salud"
    elif s < 0.8: sector = "Ciencias Sociales y Humanidades"
    elif s < 0.9: sector = "Ciencias Naturales"
    else:         sector = "Arte y Diseño"
    
    carrera = random.choice(sectores_de_estudios[sector])
    estudiantes.loc[i] = [nombre_completo, sexo, edad, rut, correo, universidad, telefono, sector, carrera, año]

In [7]:
# eliminar duplicados por rut
estudiantes.drop_duplicates(subset='RUT', inplace=True)

estudiantes.to_csv('estudiantes.csv', index=False)

# Reducir Compañias

In [8]:
companhias_elegidas = np.random.choice(np.arange(1, 145_606), size=50, replace=False)

companhias_csv = open("info/companhias.csv", "r", encoding="utf-8")
companhias_reducido_csv = open("companhias.csv", "w", encoding="utf-8")
companhias_reducido_csv.write("ID,RUT,Razon Social,Fecha de actuacion (1era firma),Fecha de registro (ultima firma),Fecha de aprobacion x SII,Anio,Mes,Comuna Tributaria,Region Tributaria,Codigo de sociedad,Tipo de actuacion,Capital,Comuna Social,Region Social\n")


for i, line in enumerate(companhias_csv):
    if i in companhias_elegidas:
        companhias_reducido_csv.write(line)


companhias_csv.close()
companhias_reducido_csv.close()

In [ ]:
def desocupacion_regiones():
    with open("info/desocupacion.csv", "r") as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        i=0
        for i, row in enumerate(reader):
            if i==0:
                index = row.index("Sep.2023")
                L = ["Desocupacion"]
                continue
            L.append(row[index])
    return L
            
            

def pib_regiones(line: str):
    n = 0
    com = False
    for i, x in enumerate(line):
        if x=='"':
            com = not com
        if com: continue
        if x==",":
            n+=1
            if n==2:
                r = line[:i]
            if n==11:
                return r+line[i:]

def limipiar_regiones():
    with open("info/PIB_regiones.csv", "r", encoding="utf-8") as file:
        lines = file.readlines()

    lines = [pib_regiones(line) for line in lines]
    desocupacion = desocupacion_regiones()
    for i in range(len(lines)):
        if i<len(lines)-1:
            lines[i] = f"{lines[i][:-1]},{desocupacion[i]}\n"
        else:
            lines[i] = f"{lines[i]},{desocupacion[i]}"


    with open("regiones.csv", "w", encoding="utf-8") as file:
        file.writelines(lines)
limipiar_regiones()

# Ofertas

In [10]:
## id
## titulo practica
## Rut companhia
## sueldo
## modalidad
## formato
companhias  = open("companhias.csv", "r", encoding="utf-8")
ofertasfile = open("ofertas.csv", "w", encoding="utf-8")

ofertasfile.write("ID,titulo practica,RUT Companhia,sueldo,modalidad,formato")
ofertas_id = []

numero_de_ofertas = 0

reader = csv.reader(companhias, delimiter=',', quotechar='"')
for i, line in enumerate(reader):
    if i==0:
        index = line.index("RUT")
        continue

    companhia_rut = line[index]

    ofertas = []
    n = max(int(abs(np.random.normal(0,10,1)[0])), 1)

    for k in range(n):
        oferta_id = np.random.randint(1, 100_000)
        while oferta_id in ofertas_id:
            oferta_id = np.random.randint(1, 100_000)
        x = random.choice(list(ofertas_de_practicas.keys()))
        y = random.choice(list(ofertas_de_practicas[x].keys()))
        oferta = random.choice(ofertas_de_practicas[x][y])
        while oferta in ofertas:
            x = random.choice(list(ofertas_de_practicas.keys()))
            y = random.choice(list(ofertas_de_practicas[x].keys()))
            oferta = random.choice(ofertas_de_practicas[x][y])
        ofertas.append(oferta)
        sueldo = int(abs(np.random.normal(0, 100_000, 1)[0])) + 200_000

        ofertasfile.write(f"\n{oferta_id},{oferta},{companhia_rut},{sueldo},{random.choice(modalidades_ofertas)},{random.choice(formatos_ofertas)}")
        numero_de_ofertas += 1
companhias.close()
ofertasfile.close()


# Postulacion

In [11]:
ofertas_disp = []

with open("ofertas.csv", "r", encoding="utf-8") as ofertas_csv:
    reader = csv.reader(ofertas_csv, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i==0:
            id_index = line.index("ID")
            rut_index = line.index("RUT Companhia")
            continue
        ofertas_disp.append((line[id_index], line[rut_index]))
## para cada estudiante agregar n postulaciones

## Rut postulante
## Rut companhia
## comuna
## id
postulacionesfile = open("postulaciones.csv", "w", encoding="utf-8")
postulacionesfile.write("ID oferta,Rut compañia,Rut postulante")

for p, rut_estudiante in enumerate(estudiantes["RUT"]):
    n = max(int(abs(np.random.normal(0,5,1)[0])), 1)
    postulaciones_ = []
    for i in range(n):
        id_oferta, rut_comp = random.choice(ofertas_disp)
        while id_oferta in postulaciones_:
            id_oferta, rut_comp = random.choice(ofertas_disp)
        postulaciones_.append(id_oferta)
        postulacionesfile.write(f"\n{id_oferta},{rut_comp},{rut_estudiante}")

postulacionesfile.close()